In [3]:
import pandas as pd
import re

# --- 1. Wczytaj plik ---
df = pd.read_excel("wsk_fin.xlsx")

# --- 2. Mapa zamian dla kolumny „wskaźnik" ---
mapping = {
    "EN":   "EN",
    "PEN":  "PEN",
    "GS":   "GS",
    "PNPM": "PNPM",
    "GS_I": "GS_I",
    "FIN":  "FIN",
    "PPO":  "PPO",
    "NP":   "NP",
    "OP":   "OP",
    "POS":  "POS",
    "CF":   "CF",
    "TC":   "TC",
    "OFE":  "OFE",
    "IP":   "IP",
    "DEPR": "DEPR",
    "IO":   "IO",
    "NWC":  "NWC",
    "C":    "C",
    "LTL":  "LTL",
    "STL":  "STL",
    "LTC":  "LTC",
    "STC":  "STC",
    "INV":  "INV",
    "REC":  "REC",
    "UP":   "UP"
}

# Usuń spacje i zamień GS (I) → GS_I jeśli takie występują
df["wskaźnik"] = df["wskaźnik"].str.replace("GS (I)", "GS_I", regex=False)

# Wyodrębnij skrót (część przed spacją lub całą zawartość jeśli nie ma spacji)
df["wskaźnik"] = df["wskaźnik"].str.extract(r'^(\w+)')[0]

# --- 3. Usuń kolumny "nazwa PKD" oraz "numer i nazwa PKD" ---
df = df.drop(columns=[c for c in ["nazwa PKD", "numer i nazwa PKD"] if c in df.columns])

# --- 4. Usuń wszystkie wiersze gdzie 'numer PKD' zawiera 'SEK' ---
df = df[~df["numer PKD"].astype(str).str.contains("SEK")]

# --- 5. Usuń kolumny 2005–2021 ---
years_to_drop = [str(y) for y in range(2005, 2022)]
df = df.drop(columns=[c for c in years_to_drop if c in df.columns])

# --- 6. Usuń wiersze gdzie numer PKD ma więcej niż jedną cyfrę po kropce ---
# Dozwolone: 01.1  35.3
# Niedozwolone: 01.11  35.23
pattern_valid = r"^\d{2}\.\d$"   # dwie cyfry, kropka, jedna cyfra

df = df[df["numer PKD"].astype(str).str.match(pattern_valid, na=False)]

print(df.head())

   numer PKD wskaźnik     2005     2006     2007     2008     2009     2010  \
72      01.1       EN      407      421      405      383      424      417   
73      01.1      PEN      349      372      375      331      359      372   
74      01.1       GS  2342.87  2624.12  2971.41  2903.12  3220.56  3516.73   
75      01.1     PNPM       bd  2196.49  2392.19  2266.51  2620.78  2887.85   
76      01.1     GS_I  2081.96  2263.84  2539.82   2419.1  2672.69  2913.21   

       2011     2012  ...     2015     2016     2017     2018     2019  \
72      425      368  ...      364      363      337      321      308   
73      359      337  ...      308      304      278      260      246   
74  3953.25  4229.45  ...  3803.48  3743.89   3867.9  3983.04  4097.06   
75  3128.72  3401530  ...  3229.18  3154.91  3304.88  3442.36  3598.17   
76  3269.35  3512.17  ...  3274.65  3219.21  3392.05  3459.63  3630.23   

       2020     2021     2022     2023     2024  
72      275      267      253 

In [6]:
# --- 7. Rozprostuj dane (unpivot) ---
df_unpivoted = df.melt(
    id_vars=["numer PKD", "wskaźnik"],
    var_name="rok",
    value_name="wartość"
)

# Konwertuj rok na liczby
df_unpivoted["rok"] = pd.to_numeric(df_unpivoted["rok"], errors="coerce")

# --- 8. Pivot tabela ---
df_pivot = df_unpivoted.pivot_table(
    index="rok",
    columns=["numer PKD", "wskaźnik"],
    values="wartość",
    aggfunc="first"
)

# Spłaszcz nazwy kolumn
df_pivot.columns = [f"{col[1]}_{col[2]}" if len(col) > 2 else f"{col[0]}_{col[1]}" for col in df_pivot.columns.values]

# Resetuj index aby "rok" stał się kolumną
df_pivot = df_pivot.reset_index()

print(df_pivot.head())
print(f"Kształt: {df_pivot.shape}")

    rok  01.1_C 01.1_CF 01.1_DEPR 01.1_EN  01.1_GS 01.1_GS_I 01.1_INV 01.1_IO  \
0  2005  202.23  310.67    128.16     407  2342.87   2081.96   636.68  224.89   
1  2006  259.93   424.8    148.08     421  2624.12   2263.84   665.76  257.74   
2  2007  318.81     575    156.09     405  2971.41   2539.82   785.61   342.5   
3  2008  341.45  486.24    167.24     383  2903.12    2419.1   877.46  421.11   
4  2009   406.9  520.94    194.38     424  3220.56   2672.69   872.72  304.86   

  01.1_IP  ... 99.0_OP 99.0_PEN 99.0_PNPM 99.0_POS 99.0_PPO 99.0_Przych  \
0   34.19  ...      bd       bd        bd       bd       bd          bd   
1   40.11  ...      bd       bd        bd       bd       bd          bd   
2   43.71  ...      bd       bd        bd       bd       bd          bd   
3   54.63  ...      bd       bd        bd       bd       bd          bd   
4   53.77  ...      bd       bd        bd       bd       bd          bd   

  99.0_REC 99.0_STC 99.0_STL 99.0_TC  
0       bd       bd    

In [13]:
df_pivot.iloc[5:10,20:40]

,01.1_Przych,01.1_REC,01.1_STC,01.1_STL,01.1_TC,01.2_C,01.2_CF,01.2_DEPR,01.2_EN,01.2_GS,01.2_GS_I,01.2_INV,01.2_IO,01.2_IP,01.2_LTC,01.2_LTL,01.2_NP,01.2_NWC,01.2_OFE,01.2_OP
5,70.43,747.37,299.22,955.38,3107.69,bd,bd,bd,17,bd,bd,bd,bd,bd,bd,bd,bd,bd,bd,bd
6,90.29,914.76,437.47,1162.93,3490.88,bd,bd,bd,15,bd,bd,bd,bd,bd,bd,bd,bd,bd,bd,bd
7,107.42,968.27,312.46,958.11,3596.32,bd,bd,bd,13,bd,bd,bd,bd,bd,bd,bd,bd,bd,bd,bd
8,88.7,872.62,323.32,939.95,3652.29,bd,bd,bd,13,bd,bd,bd,bd,bd,bd,bd,bd,bd,bd,bd
9,101.81,981.44,400.54,1073.78,3630.79,bd,bd,bd,17,bd,bd,bd,bd,bd,bd,bd,bd,bd,bd,bd


In [16]:
# --- 9. Oblicz wskaźnik "Marża netto (Net Margin)" ---
# Marża netto = NP / GS

# Konwertuj wszystkie kolumny (oprócz 'rok') na typ numeryczny
for col in df_pivot.columns:
    if col != 'rok':
        df_pivot[col] = pd.to_numeric(df_pivot[col], errors='coerce')

# Pobierz unikalne numery PKD
pkd_numbers = df_unpivoted["numer PKD"].unique()

# Dla każdego numeru PKD policz Marża netto
for pkd in pkd_numbers:
    np_col = f"{pkd}_NP"
    gs_col = f"{pkd}_GS"
    margin_col = f"{pkd}_Marża netto (Net Margin)"
    
    # Sprawdź czy obie kolumny istnieją
    if np_col in df_pivot.columns and gs_col in df_pivot.columns:
        # Policz marżę (unikając dzielenia przez zero)
        df_pivot[margin_col] = df_pivot[np_col] / df_pivot[gs_col]

print(df_pivot.head())
print(f"Kształt: {df_pivot.shape}")

    rok  01.1_C  01.1_CF  01.1_DEPR  01.1_EN  01.1_GS  01.1_GS_I  01.1_INV  \
0  2005  202.23   310.67     128.16      407  2342.87    2081.96    636.68   
1  2006  259.93   424.80     148.08      421  2624.12    2263.84    665.76   
2  2007  318.81   575.00     156.09      405  2971.41    2539.82    785.61   
3  2008  341.45   486.24     167.24      383  2903.12    2419.10    877.46   
4  2009  406.90   520.94     194.38      424  3220.56    2672.69    872.72   

   01.1_IO  01.1_IP  ...  94.1_Marża netto (Net Margin)  \
0   224.89    34.19  ...                       0.043376   
1   257.74    40.11  ...                            NaN   
2   342.50    43.71  ...                            NaN   
3   421.11    54.63  ...                            NaN   
4   304.86    53.77  ...                            NaN   

   94.2_Marża netto (Net Margin)  94.9_Marża netto (Net Margin)  \
0                            NaN                       0.043266   
1                            NaN          